In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt

In [2]:
fb_percentiles = pd.read_excel('/Users/harishprabhala/Downloads/martinchek-2012-2016-facebook-posts/fb_posts_percentiles.xlsx')

In [3]:
fb_percentiles.head()

,name,message,description,status_type,likes_count,comments_count,shares_count,year,likes_percentile,shares_percentile
0,Chief Justice Roberts Responds to Judicial Eth...,Roberts took the unusual step of devoting the ...,PAUL J. RICHARDS/AFP/Getty Images Chief Justic...,shared_story,61,27,12,2012-01-01 00:30:26,0.056,0.098
1,"With Reservations, Obama Signs Act to Allow De...",Do you agree with the new law?,"In his last official act of business in 2011, ...",shared_story,120,523,171,2012-01-01 01:08:58,0.224,0.751
2,Wishes For 2012 to Fall on Times Square,Some pretty cool confetti will rain down on Ne...,The wishes of thousands of people will flutter...,published_story,271,31,0,2012-01-01 02:00:37,0.582,0.000
3,Mitt Romney Vows to Veto Dream Act if President,NaN,"Eric Gay/AP Photo SIOUX CITY, Iowa – Mitt Romn...",shared_story,140,188,23,2012-01-01 02:35:20,0.291,0.187
4,"NY Pharmacy Shootout Leaves Suspect, ATF Agent...",The pharmacy was held up by a man seeking pres...,A shootout at a suburban New York family pharm...,shared_story,59,51,34,2012-01-01 03:36:01,0.051,0.269


In [4]:
fb_percentiles['full_text'] = fb_percentiles['name'].map(str) + ' ' + fb_percentiles['message'].map(str) + ' ' + fb_percentiles['description'].map(str)

In [5]:
bins = [0,25,50,75,100]

In [24]:
group_names = ['Quartile 1','Quartile 2','Quartile 3','Quartile 4']

In [7]:
fb_percentiles_copy = fb_percentiles[['full_text','likes_count','shares_count','likes_percentile','shares_percentile']]

In [8]:
fb_percentiles_copy.head()

,full_text,likes_count,shares_count,likes_percentile,shares_percentile
0,Chief Justice Roberts Responds to Judicial Eth...,61,12,0.056,0.098
1,"With Reservations, Obama Signs Act to Allow De...",120,171,0.224,0.751
2,Wishes For 2012 to Fall on Times Square Some p...,271,0,0.582,0.000
3,Mitt Romney Vows to Veto Dream Act if Presiden...,140,23,0.291,0.187
4,"NY Pharmacy Shootout Leaves Suspect, ATF Agent...",59,34,0.051,0.269


In [9]:
fb_percentiles_copy['likes_percentile'] = fb_percentiles_copy['likes_percentile']*100
fb_percentiles_copy['shares_percentile'] = fb_percentiles_copy['shares_percentile']*100

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [32]:
fb_percentiles_copy['likes_class'] = fb_percentiles_copy['likes_class'].fillna('Quartile 1')

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
likes_categories = pd.cut(fb_percentiles_copy['likes_percentile'], bins, labels=group_names)
fb_percentiles_copy['likes_class'] = likes_categories

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [33]:
fb_percentiles_copy.head()

,full_text,likes_count,shares_count,likes_percentile,shares_percentile,likes_class
0,Chief Justice Roberts Responds to Judicial Eth...,61,12,5.6,9.8,Quartile 1
1,"With Reservations, Obama Signs Act to Allow De...",120,171,22.4,75.1,Quartile 1
2,Wishes For 2012 to Fall on Times Square Some p...,271,0,58.2,0.0,Quartile 3
3,Mitt Romney Vows to Veto Dream Act if Presiden...,140,23,29.1,18.7,Quartile 2
4,"NY Pharmacy Shootout Leaves Suspect, ATF Agent...",59,34,5.1,26.9,Quartile 1


In [111]:
fb_percentiles_copy['likes_class'].value_counts()

Quartile 2    129675
Quartile 1    127515
Quartile 3    124126
Quartile 4    121498
Name: likes_class, dtype: int64

In [12]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

/Users/harishprabhala/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [14]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
import re
import gensim.parsing.preprocessing as process

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/harishprabhala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
lemmatizer = WordNetLemmatizer()

In [16]:
def preprocess (line):
    line = re.sub(r"http\S+", "", line)
    line = line.lower()
    line = process.remove_stopwords(line)
    line = process.strip_numeric(line)
    line = process.strip_punctuation(line)
    line = process.strip_short(line)
    line = process.strip_multiple_whitespaces(line)
    tokens = sent_tokenize(line)
    stemmed = [w for w in tokens if lemmatizer.lemmatize(w)] 
    return ' '.join(stemmed)

In [17]:
text = []
for w in fb_percentiles_copy['full_text']:
    text.append(preprocess(w))

In [34]:
X,y = text, fb_percentiles_copy['likes_class']

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [149]:
vectorizer = CountVectorizer(lowercase=True,stop_words='english')
X_1 = vectorizer.fit_transform(X)

In [150]:
X_1.shape, y.shape

((502814, 142431), (502814,))

In [36]:
score = chi2(X_1,y)

In [161]:
X_new = SelectKBest(chi2, k=26000).fit_transform(X_1, y)

In [162]:
X_new.shape

(502814, 26000)

In [49]:
from sklearn.linear_model import LogisticRegression

In [163]:
X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, test_size=0.33)

In [164]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((336885, 26000), (165929, 26000), (336885,), (165929,))

In [138]:
logit=LogisticRegression(multi_class='multinomial', solver='sag',C=10)
logit.fit(X_train,Y_train)
print ('Accuracy of test data: ',(accuracy_score(logit.predict(X_test), Y_test)))
print ('Accuracy of training data: ',(accuracy_score(logit.predict(X_train), Y_train)))
print("\n")
print ('Classification report summary of Multinomial Logistic Regression')
print(classification_report(logit.predict(X_test), Y_test))

Accuracy of test data:  0.373762271815
Accuracy of training data:  0.392682963029


Classification report summary of Multinomial Logistic Regression
             precision    recall  f1-score   support

 Quartile 1       0.50      0.41      0.45     51110
 Quartile 2       0.29      0.31      0.30     40521
 Quartile 3       0.19      0.30      0.23     27019
 Quartile 4       0.51      0.43      0.47     47279

avg / total       0.40      0.37      0.38    165929



In [63]:
from sklearn.metrics import confusion_matrix

In [65]:
y_true = np.array(Y_test)
y_pred = np.array(logit.predict(X_test))

In [67]:
conf_matrix = confusion_matrix(y_true, y_pred)

In [112]:
from sklearn.metrics import f1_score

In [115]:
f1_score(y_true, y_pred, average='weighted') 

0.38247767877828526

In [165]:
clf2=MultinomialNB(alpha=0.01)
clf2.fit(X_train,Y_train)
print ('Accuracy of test data: ',(accuracy_score(clf2.predict(X_test), Y_test)))
print ('Accuracy of training data: ',(accuracy_score(clf2.predict(X_train), Y_train)))
print("\n")
print ('Classification report summary of MultinomialDB with alpha = 0.01:')
print(classification_report(clf2.predict(X_test), Y_test))

Accuracy of test data:  0.395090671311
Accuracy of training data:  0.446564851507


Classification report summary of MultinomialDB with alpha = 0.01:
             precision    recall  f1-score   support

 Quartile 1       0.50      0.45      0.47     47092
 Quartile 2       0.28      0.34      0.31     34580
 Quartile 3       0.23      0.33      0.27     28557
 Quartile 4       0.58      0.42      0.49     55700

avg / total       0.43      0.40      0.41    165929

